In [1]:
!pip install python-chess
!pip install psutil
!pip install google-cloud-storage

import numpy as np
import pandas as pd
import glob
import os
import chess


In [ ]:
import psutil
memory = psutil.virtual_memory()
print(f"Total RAM: {memory.total / (1024 ** 3):.2f} GB")
!nvidia-smi


Total RAM: 102.18 GB
Fri Oct  6 11:07:38 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P8              10W /  70W |      2MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/content/gleaming-modem-400906-3780658b0f47.json"
from google.cloud import storage

def download_from_bucket(blob_name, path_to_save_to, bucket_name):
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.download_to_filename(path_to_save_to)
    print(f"File {blob_name} downloaded to {path_to_save_to}.")

# Use the function to download your file:
download_from_bucket("concatenated_array2.npy", "/content/concatenated_array2.npy", "chess_data_all")
download_from_bucket("concatenated_array1.npy", "/content/concatenated_array1.npy", "chess_data_all")


File concatenated_array2.npy downloaded to /content/concatenated_array2.npy.
File concatenated_array1.npy downloaded to /content/concatenated_array1.npy.


In [2]:
bitboard_dataset=np.load("/content/concatenated_array1.npy")
labels=np.load("/content/concatenated_array2.npy")

In [4]:
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.layers import Input, Conv2D, Conv3D, Flatten, Dense, Concatenate
from tensorflow.keras.activations import relu
import tensorflow as tf
from tensorflow.keras.layers import DepthwiseConv2D


# ... Assuming bitboard_dataset and labels are defined ...

bitboard_dataset, labels = shuffle(bitboard_dataset, labels)
X_train, X_test, y_train, y_test = train_test_split(bitboard_dataset, labels, test_size=0.2, random_state=42)


# Input layer
input_layer_2d = layers.Input(shape=(16,8, 8))  # for 2D convolutions
input_layer_3d = layers.Reshape((16, 8, 8, 1))(input_layer_2d)  # for 3D convolutions


In [5]:

# Branch 1: Depthwise Convolution
branch1 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(input_layer_2d)
branch4 = Conv2D(filters=5, kernel_size=(8, 8), activation=relu)(branch1)
branch1 = Flatten()(branch1)  # Flatten branch1
branch4 = Flatten()(branch4)
# Branch 2: Conv3D
branch2 = Conv3D(filters=16, kernel_size=(5, 5, 5), padding='valid', activation=relu)(input_layer_3d)
branch2 = Conv3D(filters=8, kernel_size=(3, 3, 3), padding='valid', activation=relu)(branch2)
branch2 = Flatten()(branch2)  # Flatten branch2

# Branch 3: Conv3 (Only to the 15th channel)
branch3_input = tf.gather(input_layer_2d, [13,14], axis=3)  # Extract the 15th channel
branch3 = DepthwiseConv2D(kernel_size=(1, 1), depth_multiplier=1, activation=relu)(branch3_input)
branch3 = Conv2D(filters=10, kernel_size=(8,8), activation=relu)(branch3)
branch3 = Conv2D(filters=2, kernel_size=(1, 1), activation=relu)(branch3)
branch3 = Flatten()(branch3)  # Flatten branch3

concat = Concatenate()([branch1, branch2, branch3,branch4])

# Dense layers
dense1 = Dense(1024, activation='relu')(concat)
dense2 = Dense(512, activation='relu')(dense1)
dense3 = Dense(128, activation='relu')(dense2)
dense4 = Dense(64, activation='relu')(dense3)
output_layer = Dense(1)(dense4)  # No activation function for regression

# Create the model
model = Model(inputs=input_layer_2d, outputs=output_layer)
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mae', 'mape'])

# Train the model
model.fit(X_train, y_train, epochs=200, batch_size=250, validation_split=0.1)

# Evaluate the model
loss, mae, mape = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Mean Absolute Error:", mae)
print("Mean Absolute Percentage Error:", mape)

Epoch 1/200
5707/5707 [==============================] - 56s 7ms/step - loss: 7234.6514 - mae: 33.7384 - mape: 2129335808.0000 - val_loss: 6767.1069 - val_mae: 34.5603 - val_mape: 2744451072.0000
Epoch 2/200
5707/5707 [==============================] - 37s 7ms/step - loss: 6058.1519 - mae: 31.7933 - mape: 2220264960.0000 - val_loss: 6137.2271 - val_mae: 32.7741 - val_mape: 2560189952.0000
Epoch 3/200
5707/5707 [==============================] - 39s 7ms/step - loss: 5163.1772 - mae: 30.2514 - mape: 2086460544.0000 - val_loss: 5672.9482 - val_mae: 31.1693 - val_mape: 2320307200.0000
Epoch 4/200
5707/5707 [==============================] - 37s 6ms/step - loss: 4434.1006 - mae: 28.8616 - mape: 1952464384.0000 - val_loss: 5415.7832 - val_mae: 30.1318 - val_mape: 2026812672.0000
Epoch 5/200
5707/5707 [==============================] - 37s 7ms/step - loss: 3869.8909 - mae: 27.7487 - mape: 1820376320.0000 - val_loss: 5467.5361 - val_mae: 31.9488 - val_mape: 2381706496.0000
Epoch 6/200
5707/570

KeyboardInterrupt: ignored